In [ ]:
import math
import numpy as np
import pandas as pd
from pandas import Series
#
from numpy import log, mean, sqrt, where, std, exp, sign
from scipy import linalg as LA
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
#
import matplotlib.pyplot as plt
import time
#
from scipy.signal import savgol_filter
#
from scipy import stats
from scipy.stats import entropy
from scipy.stats import vonmises

In [ ]:
from numpy import finfo
epsilon = finfo(float).eps
print(epsilon)

In [ ]:
#
# apply savgol_filter for records whose length is longer than window_length
# if the record length is smaller than window_length, do not apply savgol_filter
#
def condition_length(x):
    #
    window_length = 25
    poly_degree = 5
    #
    if (len(x) >= window_length):
        return savgol_filter(x, window_length, poly_degree)
    else:
        return x

In [ ]:
#
# basic functions
#
def get_displacement(x_1, y_1, x_2, y_2):
    # compute the displacement between two points
    return sqrt((x_1-x_2)**2+(y_1-y_2)**2)

def get_displacements(x, y):
    #
    # inputs
    # x: list of x coordinate
    # y: list of y coordinate
    #
    # results
    # displacements: list of displacements between position at t(i-1) and t(i)
    #
    n_datapoints = len(x)
    displacements = np.array([get_displacement(x[i-1], y[i-1], x[i], y[i]) for i in range(1, n_datapoints)])
    return displacements

def get_net_displacement(x, y):
    n_datapoints = len(x)
    x_0 = x[0]
    y_0 = y[0]
    x_f = x[n_datapoints-1]
    y_f = y[n_datapoints-1]
    net_displacement = get_displacement(x_0, y_0, x_f, y_f)
    return net_displacement

def get_d_traveled(x, y):
    #
    # inputs
    # x: list of x coordinate
    # y: list of y coordinate
    #
    # results
    # d_traveled: sum of list of displacements
    #    
    n_datapoints = len(x)
    d_traveled = sum(get_displacements(x, y)) 
    return d_traveled

In [ ]:
#
# feature 1: efficiency and straightness
#
def get_efficiency(x, y):
    #
    # inputs
    # traj_x: list of x coordinate
    # traj_y: list of y coordinate
    #
    # results
    # efficiency: relevant to trajectory linearity; may be useful for detecting directed motion
    #       
    n_datapoints = len(x)
    upper = get_displacement(x[n_datapoints-1], y[n_datapoints-1], x[0], y[0])**2
    displacements_sq = get_displacements(x, y)**2
    lower = (n_datapoints-1)*sum(displacements_sq)
    efficiency = upper/lower
    return efficiency

def get_straightness(x, y):
    #
    # inputs
    # x: list of x coordinate
    # y: list of y coordinate
    #
    # results
    # straightness: relevant to average direction change between subsequent steps; may be useful for detecting directed motion
    #         
    n_datapoints = len(x)
    x_0 = x[0]
    y_0 = y[0]
    x_f = x[n_datapoints-1]
    y_f = y[n_datapoints-1]     
    upper = get_displacement(x_0, y_0, x_f, y_f)
    lower = get_d_traveled(x, y)
    straightness = upper/lower
    return straightness


In [ ]:
#
# feature 2: turning angle 
#
# entropy of turning angle 
# see:
# Liu et al. PRE 2017
# Establishing the kinetics of ballistic-to-diffusive transition using directional statistics
# Appendix A: Determining theta from trajectory
#
def get_turning_angle_measures(traj_x, traj_y, tau_frame, binwidth):
    #
    # compute turning angle
    #    
    # traj_x = record_raw['relative_x']
    # traj_y = record_raw['relative_y']
    traj_x = pd.Series(traj_x)
    traj_y = pd.Series(traj_y)
    relative_x = traj_x[::tau_frame] # every tau-th row
    relative_y = traj_y[::tau_frame] # every tau-th row  
    relative_x = relative_x.reset_index(drop=True) # reset row index
    relative_y = relative_y.reset_index(drop=True) # reset row index
    turning_angle = []
    for i in range (1, len(relative_x)-1):
        # print(i)
        diff_x1 = relative_x[i]-relative_x[i-1]
        diff_x2 = relative_x[i+1]-relative_x[i]
        diff_y1 = relative_y[i]-relative_y[i-1]
        diff_y2 = relative_y[i+1]-relative_y[i] 
        # compute k1 and k2
        k1 = 0 # diff_x1 > 0 and diff_y1 > 0
        if (diff_x1 >= 0):
            if (diff_y1 >= 0):
                k1 = 0
            else:
                k1 = 2
        if (diff_x1 < 0):
            k1 = 1
        k2 = 0
        if (diff_x2 >= 0):
            if (diff_y2 >= 0):
                k2 = 0
            else:
                k2 = 2
        if (diff_x2 < 0):
            k2 = 1    
        # compute phi_1 and phi_2
        # arc tangent of y/x in radians
        phi_1 = k1*np.pi + math.atan2(diff_y1, diff_x1) # 0, ..., 2*np.pi
        phi_2 = k2*np.pi + math.atan2(diff_y2, diff_x2) # 0, ..., 2*np.pi
        # compute m
        m = 0
        phi_diff = abs(phi_2-phi_1)
        if (phi_diff < np.pi):
            m = 0
        if (phi_diff > np.pi):
            if (phi_2 > phi_1):
                m = -1
            if (phi_2 < phi_1):
                m = 1
        # compute theta
        theta_i = 2*m*np.pi+phi_2-phi_1  # -np.pi, ..., np.pi
        turning_angle.append(theta_i)
    #
    # compute entropy
    #
    # epsilon_p = 0.0001
    # epsilon_q = 0.0001
    epsilon_p = epsilon
    epsilon_q = epsilon 
    # create relative frequency histogram
    turning_angle_deg = [(x/np.pi)*180.0 for x in turning_angle] # degree
    x_max = 180 # deg
    x_min = -180 # deg
    #
    bin_list = np.arange(x_min, x_max, binwidth) 
    hist_p, edges_p = np.histogram(turning_angle_deg, bins=bin_list)
    freq_p = hist_p/float(hist_p.sum())
    freq_p += epsilon_p
    pk = np.reshape(freq_p, -1)
    base = len(bin_list) # normalized entropy
    entropy_p = entropy(pk, base=base)        
    #
    turning_angle_avg = np.mean(turning_angle)
    turning_angle_std = np.std(turning_angle)
    turning_angle_entropy = entropy_p
    #
    return turning_angle_avg, turning_angle_std, turning_angle_entropy


In [ ]:
#
# feature 3: step size
#
def get_stepsize(traj_x, traj_y, tau_frame):
    traj_x = pd.Series(traj_x)
    traj_y = pd.Series(traj_y)
    x_sampled = traj_x[::tau_frame] # every tau-th frames
    y_sampled = traj_y[::tau_frame] # every tau-th frames
    x_sampled = x_sampled.reset_index(drop=True) # reset index
    y_sampled = y_sampled.reset_index(drop=True) # reset index
    #
    n_datapoints = len(x_sampled)
    stepsize = np.array([get_displacement(x_sampled[i], y_sampled[i], x_sampled[i-1], y_sampled[i-1]) for i in range(1, n_datapoints)])
    return stepsize

In [ ]:
#
# additional information
#
def check_entering(x, y, r_contact):
    N = len(x)
    #
    x_0 = x[0]
    y_0 = y[0] 
    dij_0 = np.sqrt(x_0*x_0+y_0*y_0)
    #
    if (dij_0 >= (r_contact-0.05)):
        return 1
    else:
        return 0
#
def check_exiting(x, y, r_contact):
    N = len(x)
    #
    x_f = x[N-1]
    y_f = y[N-1] 
    dij_f = np.sqrt(x_f*x_f+y_f*y_f)
    #
    if (dij_f >= (r_contact-0.05)):
        return 1
    else:
        return 0
#

def estimate_von_mises(data):
    res = vonmises.fit(data, fscale=1)
    mu = res[1]
    k = res[0]
    return mu, k
#

def estimate_von_mises_mle(data):
    #
    # based on maximum likelihood estimation presented in Banerjee et al (2005) 
    # see:
    # https://stats.stackexchange.com/questions/18692/estimating-kappa-of-von-mises-distribution
    # https://stackoverflow.com/questions/47746500/estimating-parameters-of-von-mises-distribution-scipy-inconsistent-answers
    #
    dimension = 2
    mu = np.arctan2(sum(np.sin(data)), sum(np.cos(data)))
    A = sum(np.cos(data))*np.cos(mu)*(1/len(data))+ sum(np.sin(data))*np.sin(mu)*(1/len(data))
    k = A*(dimension-A**2)/(1-A**2)
    return mu, k
#

In [ ]:
def get_motion_type(entropy, efficiency):
    #
    # threshold_values
    # firstly, characterize ballistic motion by turning angle entropy H <= 0.26
    # secondly, characterize confined motion by efficiency E <= 0.09
    # lastly, characterize sub-ballistic motion for the parameter space does not belong to either ballistic or confined motions
    #
    entropy_threshold = 0.26
    efficiency_threshold = 0.09
    motion_type = "" 
    #
    if (entropy <= entropy_threshold):
        # ballistic motion
        motion_type = "B"
    else:
        if (efficiency <= efficiency_threshold):
            # confined motion
            motion_type = "C"
        else:
            # sub-ballistic
            motion_type = "S"            
    #
    return motion_type

In [ ]:
def get_tc_assumed(r_contact, v_ini, angle_i, angle_j, angle_ij):
    upper = 2*r_contact*abs(np.cos(angle_i)-np.cos(angle_j))
    lower = v_ini*(1.0-np.cos(angle_ij))
    if (lower <= epsilon):
        lower = epsilon
    tc_assumed = upper/lower
    return tc_assumed

In [ ]:
def extract_features(scenario_name, fps):
    #
    # t_temp = time.localtime()
    # t_start = time.strftime("%H:%M:%S", t_temp)
    #
    # read contact record
    #
    dt = 1.0/fps
    readfile_name = "contact_"+str(fps)+"fps_"+scenario_name+".txt"
    print ("read ", readfile_name)
    r_contact = 2.0
    area_circle = np.pi*r_contact**2
    #
    columns = [
    'frame', 'id_i', 'id_j', 'id_ij', 'degree_i', 'x_i', 'y_i', 'deltax_raw', 'deltay_raw', 'd_ij', 
    'angle_i', 'angle_j', 'angle_ij', 'v_i', 'v_j', 'v_ij', 'polarization']
    #
    record_raw = pd.read_csv(readfile_name, sep= "\t", names = columns) 
    record_raw = record_raw.sort_values(['id_i', 'id_j', 'frame'], ascending=[True, True, True])
    record_raw = record_raw.drop(record_raw[record_raw.d_ij > r_contact].index)
    #
    pairs_0 = record_raw['id_ij'].unique()
    n_pairs0 = len(pairs_0)
    #
    record_groupby = record_raw.groupby(['id_ij'], group_keys=False, as_index=False).agg(list)
    #
    # remove id_ij recrods containing datapoints less than window_length
    # https://stackoverflow.com/questions/37335598/how-to-get-the-length-of-a-cell-value-in-pandas-dataframe
    #
    # window_length_min is set as 10 based on literature
    # Briane et al. (2016) An adaptive statistical test to detect non Brownian diffusion from particle trajectories
    # Briane et al. (PRE 2018) Statistical analysis of particle trajectories in living cells
    # 
    window_length = 25
    window_length_min = 8
    if (fps == 25):
        window_length_min = 12  
    record_groupby = record_groupby.drop(record_groupby[record_groupby.d_ij.str.len() < window_length_min].index)    
    #
    record_groupby['delta_x']=record_groupby['deltax_raw'].apply(condition_length)
    record_groupby['delta_y']=record_groupby['deltay_raw'].apply(condition_length)
    record_groupby = record_groupby.drop(columns=['id_i', 'id_j'])   
    #
    record_groupby = record_groupby.drop(['deltax_raw', 'deltay_raw'], axis=1)
    record_groupby = record_groupby.drop(['x_i', 'y_i'], axis=1)
    record_groupby = record_groupby.reindex(
    columns=['id_ij', 'frame', 'degree_i', 'delta_x', 'delta_y', 'd_ij', 'angle_i', 'angle_j', 'angle_ij', 'v_i', 'v_j', 'v_ij'])
    #
    record_groupby['delta_x']=record_groupby['delta_x'].apply(lambda x: np.round(x, decimals=3))
    record_groupby['delta_y']=record_groupby['delta_y'].apply(lambda x: np.round(x, decimals=3))
    record_groupby['angle_i']=record_groupby['angle_i'].apply(lambda x: np.round(x, decimals=3))
    record_groupby['angle_j']=record_groupby['angle_j'].apply(lambda x: np.round(x, decimals=3))
    record_groupby['angle_ij']=record_groupby['angle_ij'].apply(lambda x: np.round(x, decimals=3))
    record_groupby['v_i']=record_groupby['v_i'].apply(lambda x: np.round(x, decimals=3))
    record_groupby['v_j']=record_groupby['v_j'].apply(lambda x: np.round(x, decimals=3))
    record_groupby['v_ij']=record_groupby['v_ij'].apply(lambda x: np.round(x, decimals=3))    
    #
    pairs_1 = record_groupby['id_ij'].unique()
    n_pairs1 = len(pairs_1)  
    #
    # create dataframe for summary
    summary = pd.DataFrame(columns=[
        'id_ij', 
        'frame_start',
        'frame_end', 
        'n_datapoints',     
        'heading_i',
        'heading_iavg',
        'heading_istd',
        'heading_j',
        'heading_javg',
        'heading_jstd',
        'alpha_ini',
        'alpha_avg',
        'alpha_std',         
        'v_ini',
        'v_avg',
        'v_std',
        'n_neighbors_avg', 
        'n_neighbors_std',
        'entering_circle',
        'exiting_cricle',
        'tc_data',
        'tc_assumed',         
        'efficiency',
        'net_displacement',
        'd_traveled',        
        'straightness',       
        'turning_angle_avg',  
        'turning_angle_std',  
        'turning_angle_entropy',   
        'stepsize_avg',
        'stepsize_std',        
        'motion_type'
        ])
    #
    # compute features/metrics row by row
    #
    for index, row in record_groupby.iterrows():
        #
        frame = row['frame']
        frame_start = row['frame'][0]          # contact interaction start time (frame)
        frame_end = row['frame'][-1]           # contact interaction end time (frame)
        relative_x = row['delta_x']
        relative_y = row['delta_y']
        relative_v = row['v_ij']
        degree = row['degree_i']               # the number of neighbors within contact radius
        #
        angle_i = row['angle_i']
        angle_j = row['angle_j']
        angle_ij = row['angle_ij']             # contact angle between individuals in contact
        #
        heading_i = row['angle_i'][0]          # walking direction of individiual i at the beginning of contact interaction
        heading_iavg = np.mean(row['angle_i'])
        heading_istd = np.std(row['angle_i'])  
        #
        heading_j = row['angle_j'][0]          # walking direction of individiual j at the beginning of contact interaction
        heading_javg = np.mean(row['angle_j'])
        heading_jstd = np.std(row['angle_j'])          
        #
        alpha_ini = row['angle_ij'][0]          # contact angle at the beginning of contact interaction
        alpha_avg = np.mean(row['angle_ij'])    # average of contact angle
        alpha_std = np.std(row['angle_ij'])     # standard deviation of contact angle        
        #
        # basic values
        #
        # the number of trajectory data points 
        # represented the number of elements in x coordinate array/list
        # in oother studies, this is denoted as "total length of the path" or "trajectory length"
        n_datapoints = len(relative_x)
        #
        net_displacement = get_net_displacement(relative_x, relative_y)
        # distance traveled within the contact radius
        d_traveled = get_d_traveled(relative_x, relative_y)
        #
        # contact angle
        alpha_ini = angle_ij[0]
        #
        # the number of neighbors
        n_neighbors_avg = np.mean(degree)
        n_neighbors_std = np.std(degree)
        #
        # feature 1: efficiency and straightness
        #
        efficiency = get_efficiency(relative_x, relative_y)
        straightness = get_straightness(relative_x, relative_y)
        #
        # feature 2: turning angle
        #
        binwidth_turning_angle = 15 # deg
        tau_turning_angle = 2 # frames (default 2)
        if (fps == 25):
            tau_turning_angle = 3 # frames (default 3)
        #     
        turning_angle_avg, turning_angle_std, turning_angle_entropy = get_turning_angle_measures(relative_x, relative_y, tau_turning_angle, binwidth_turning_angle)
        #
        # feature 3: step length
        #
        stepsize = get_stepsize(relative_x, relative_y, tau_turning_angle)
        stepsize_avg = np.average(stepsize)
        stepsize_std = np.std(stepsize)        
        #
        # von mises distribution parameter estimation
        #
        # mu_ij, k_ij = estimate_von_mises_mle(angle_ij) # estimate von mises distribution parameters        
        #
        # empirical contact duration and contact speed
        #
        tc_data = (n_datapoints-1)*dt # measured from experimental data 
        v_ini = relative_v[0]     
        v_avg = np.mean(relative_v)
        v_std = np.std(relative_v)
        #
        # motion type classification
        #
        motion_type_i = get_motion_type(turning_angle_entropy, efficiency)
        #
        # additional infomation
        #
        tc_assumed = get_tc_assumed(r_contact, v_ini, angle_i[0], angle_j[0], angle_ij[0]) # estimated based on ballistic motion assumption
        entering_circle = check_entering(relative_x, relative_y, r_contact)
        exiting_circle = check_exiting(relative_x, relative_y, r_contact) 
        #
        # append records row by row
        # 
        summary.loc[index] = pd.Series({           
            'id_ij': row['id_ij'], 
            'frame_start': int(frame_start),
            'frame_end': int(frame_end), 
            'n_datapoints': n_datapoints,           
            'heading_i': round(heading_i, 3),
            'heading_iavg': round(heading_iavg, 3),
            'heading_istd': round(heading_istd, 3),
            'heading_j': round(heading_j, 3),
            'heading_javg': round(heading_javg, 3),
            'heading_jstd': round(heading_jstd, 3),
            'alpha_ini': round(alpha_ini, 3),
            'alpha_avg': round(alpha_avg, 3),
            'alpha_std': round(alpha_std, 3),               
            'v_ini': v_ini.round(3), 
            'v_avg': v_avg.round(3), 
            'v_std': v_std.round(3),
            'n_neighbors_avg': round(n_neighbors_avg, 2), 
            'n_neighbors_std': round(n_neighbors_std, 2),              
            'entering_circle': int(entering_circle),
            'exiting_cricle': int(exiting_circle),   
            'tc_data': round(tc_data, 3),  
            'tc_assumed': round(tc_assumed, 3),              
            'efficiency': efficiency.round(3),  
            'net_displacement': net_displacement.round(3),
            'd_traveled': round(d_traveled, 3),            
            'straightness': straightness.round(3),
            'turning_angle_avg': round(turning_angle_avg, 3),    
            'turning_angle_std': round(turning_angle_std, 3),  
            'turning_angle_entropy': round(turning_angle_entropy, 3),
            'stepsize_avg': round(stepsize_avg, 3),
            'stepsize_std': round(stepsize_std, 3),              
            'motion_type': motion_type_i,
        })
    #
    # sort dataframe by contact interaction end and start frames
    #
    summary.sort_values(by=['frame_end', 'frame_start'])         
    #
    # write features for classification inputs
    #
    filename_write = 'traj-classification-features_'+scenario_name+'.txt'
    summary.to_csv(filename_write, index=False)   
    #
    # print statistics
    #
    print(str(n_pairs0)+"\t"+str(n_pairs1))  


In [ ]:
scenario_name_set = ["uni-01", "uni-02", "uni-03", "uni-04", "uni-05", 
                     "uni-06", "uni-07", "uni-08", "uni-09"]
#
fps_values = [25]
#
for i in range(0, len(scenario_name_set)):
    t_temp = time.localtime()
    t_start = time.strftime("%H:%M:%S", t_temp)
    print(t_start)
    #
    # print(scenario_name_set[i], fps_values[i])
    extract_features(scenario_name_set[i], fps_values[0])
    #
    t_temp = time.localtime()
    t_end = time.strftime("%H:%M:%S", t_temp)
    print(t_end)    
    print(t_start)